In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

np.random.seed(2)

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import itertools

from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


sns.set(style='white', context='notebook', palette='deep')

Using TensorFlow backend.


In [6]:
X = pd.read_csv("/kaggle/input/dataquest-2-2019/train.csv")
test = pd.read_csv("/kaggle/input/dataquest-2-2019/test.csv")

In [12]:
X.head(3)

,id,Gender,Country,Age,Income,Marital-Status,workclass,credit-score,Learning,Tenure,Balance,Products,Education,Occupation,Target
0,0,Female,Australia,53.0,20000.0,Separated,Local-gov,18396.0,8.0,7.0,100000.0,6.0,Masters,Adm-clerical,0
1,1,Female,Canada,28.0,100000.0,Separated,Private,10886.0,4.0,4.0,101300.0,5.0,HS-Grad,Prof-specialty,0
2,2,Female,Australia,35.0,200000.0,Never-married,Private,71561.0,4.0,4.0,104000.0,3.0,HS-Grad,Prof-specialty,0


In [13]:
y = X["Target"]
y.head(3)


0    0
1    0
2    0
Name: Target, dtype: int64

In [14]:
del X["Target"]

In [11]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14000 entries, 0 to 13999
Data columns (total 15 columns):
id                14000 non-null int64
Gender            14000 non-null object
Country           14000 non-null object
Age               14000 non-null float64
Income            14000 non-null float64
Marital-Status    14000 non-null object
workclass         14000 non-null object
credit-score      14000 non-null float64
Learning          12593 non-null float64
Tenure            14000 non-null float64
Balance           14000 non-null float64
Products          12608 non-null float64
Education         14000 non-null object
Occupation        14000 non-null object
Target            14000 non-null int64
dtypes: float64(7), int64(2), object(6)
memory usage: 1.6+ MB


In [16]:
traindf=X
testdf=test
del X
del test

In [18]:
del traindf["Learning"]
del testdf["Products"]

In [19]:
del traindf["Products"]
del testdf["Learning"]

In [24]:
del traindf["id"]
del testdf["id"]

In [25]:
print(np.shape(traindf))
print(np.shape(testdf))

(14000, 11)
(5963, 11)


In [26]:
traindf.head(3)

,Gender,Country,Age,Income,Marital-Status,workclass,credit-score,Tenure,Balance,Education,Occupation
0,Female,Australia,53.0,20000.0,Separated,Local-gov,18396.0,7.0,100000.0,Masters,Adm-clerical
1,Female,Canada,28.0,100000.0,Separated,Private,10886.0,4.0,101300.0,HS-Grad,Prof-specialty
2,Female,Australia,35.0,200000.0,Never-married,Private,71561.0,4.0,104000.0,HS-Grad,Prof-specialty


In [27]:
testdf.head(3)

,Gender,Country,Age,Income,Marital-Status,workclass,credit-score,Tenure,Balance,Education,Occupation
0,Female,Australia,38.0,180000.0,Never-married,Private,151031.0,4.0,107011.0,HS-Grad,Prof-specialty
1,Female,France,22.0,50000.0,Separated,Private,50089.0,4.0,101798.0,HS-Grad,Prof-specialty
2,Male,Australia,33.0,300000.0,Separated,Private,283552.0,4.0,110000.0,HS-Grad,Prof-specialty


In [29]:
testdf = pd.get_dummies(testdf, columns = ["Gender","Country","Marital-Status","workclass","Education","Occupation"],
                             prefix=[ "Sex","Country","Status","workclass","Education","Occupation"])

In [30]:
traindf = pd.get_dummies(traindf, columns = ["Gender","Country","Marital-Status","workclass","Education","Occupation"],
                             prefix=[ "Sex","Country","Status","workclass","Education","Occupation"])

In [31]:
print(np.shape(traindf))
print(np.shape(testdf))

(14000, 41)
(5963, 41)


In [32]:
traindf.head(3)

,Age,Income,credit-score,Tenure,Balance,Sex_Female,Sex_Male,Country_Australia,Country_Canada,Country_France,...,Education_Masters,Education_Preschool,Education_Some-college,Occupation_Adm-clerical,Occupation_Craft-repair,Occupation_Exec-managerial,Occupation_Farming-fishing,Occupation_Prof-specialty,Occupation_Protective-serv,Occupation_Tech-support
0,53.0,20000.0,18396.0,7.0,100000.0,1,0,1,0,0,...,1,0,0,1,0,0,0,0,0,0
1,28.0,100000.0,10886.0,4.0,101300.0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,35.0,200000.0,71561.0,4.0,104000.0,1,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0


In [33]:
import xgboost as xgb

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [49]:
seed = 2
test_size = 0.2

In [50]:
X_train, X_test, y_train, y_test = train_test_split(traindf, y, test_size=test_size, random_state=seed)

In [93]:
from numpy import loadtxt
from xgboost import XGBClassifier


In [106]:
model = XGBClassifier( n_estimators=100, learning_rate=0.003000012)
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.003000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)

In [111]:
# make predictions for test data
y_pred = model.predict(X_test).astype(int)
predictions = [round(value) for value in y_pred]

In [112]:
print(np.shape(y_pred))

(2800,)


In [113]:
accuracy = accuracy_score(y_test, predictions)#.astype(int)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 81.07%


In [114]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/dataquest-2-2019/sample_submission.csv
/kaggle/input/dataquest-2-2019/test.csv
/kaggle/input/dataquest-2-2019/train.csv


In [115]:

predictions = pd.DataFrame(y_pred, columns=['Target])
test = pd.read_csv(os.path.join('/kaggle/input/dataquest-2-2019/', 'test.csv'))
predictions = pd.concat((test.iloc[:, 0], predictions), axis = 1)
predictions.to_csv('my_output.csv', sep=",", index = False)

In [81]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers
import keras as K

In [91]:
#K.optimizers.Adamax(learning_rate=0.0002, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [92]:
model.fit(traindf, y, epochs=5, batch_size=16)

Epoch 1/5
14000/14000 [==============================] - 3s 233us/step - loss: 0.5221 - accuracy: 0.7802
Epoch 2/5
14000/14000 [==============================] - 3s 212us/step - loss: 0.5131 - accuracy: 0.7810
Epoch 3/5
14000/14000 [==============================] - 3s 210us/step - loss: 0.5124 - accuracy: 0.7810
Epoch 4/5
14000/14000 [==============================] - 3s 224us/step - loss: 0.5109 - accuracy: 0.7810
Epoch 5/5
14000/14000 [==============================] - 3s 219us/step - loss: 0.5106 - accuracy: 0.7810
